# Example notebook for running jobs on MlSpace

## Важные детали
0. В данном ноутбуке используется client_lib для запуска задач - эта библиотека работает только на инстансах mlspace. Поэтому алгоритм запуска задач при помощи client_lib следующий - необходимо создать бесплатный инстанс с cpu, и на нем выполнить дальнейшие шаги. Другой вариант - запуск при помощи api - также требует созданного инстанса для получения ключей для авторизации, но при этом запустить задачу при помощи api можно удаленно, подробнее это описано [здесь](https://github.com/sbercloud-ai/aicloud-examples/blob/master/public-api-example/public-api-examples-notebook.ipynb).
1. Задача запускается внутри контейнера, который собирается либо на основе одного из [доступных образов](https://mlspace.aicloud.sbercloud.ru/mlspace/datahub-catalog), либо на основе своего образа - он собирается при помощи базового образа и requirements.txt.
2. При запуске задачи из контейнера нет доступа к интернету, поэтому все библиотеки нужно установить на этапе сборки.
3. Контейнер и инстанс, из которого запускается задача, имеют общее дисковое пространство. Поэтому если для задачи необходим датасет/модель/что-то еще, то их можно заранее сохранить на диск и загрузить сохраненную копию при обучении.
4. Важно: задачи в контейнере запускаются от имени другого пользователя (не совпадает с пользователем в инстансе). Поэтому при сохранении данных в домашней директории на инстансе необходимо копировать их в директорию пользователя контейнера (актуально для huggingface, т. к. часть его файлов кэшируется в ~/).
5. При помощи client_lib.Job на данный момент лучше запускать только .py файлы (.sh скрипты можно обернуть в простую обертку (например, os.system('run.sh'))) Также можно запускать и .sh скрипты, но на данный момент при запуске они автоматически будут выполняться в параллельном режиме при запуске на нескольких нодах/инстансах.
6. Текущий статус задачи и логи можно смотреть не только при помощи client_lib, но и в разделе Environments->Задачи и окружения->Задачи.

Остальные детали и пример запуска задачи (на примере эксперимента из репозитория active_learning) представлены дальше. Больше информации по client_lib можно найти по ссылкам ([документация](https://docs.sbercloud.ru/aicloud/mlspace/concepts/client-lib.html), [примеры](https://github.com/sbercloud-ai/aicloud-examples/blob/master/quick-start/job_launch_tf2/quick-start-v100.ipynb)).

In [1]:
try:
    import client_lib
except ImportError:
    raise RuntimeError("Скрипт не предназначен для запуска вне кластера")

# Preprocessing part (for active learning)
Use a special script for loading and caching all data - active_learning/examples/cache_necessary_files/run.sh. Before calling this script, set used dataset and model in run.sh. After set cache_dir in your training config.

In [2]:
import os

In [3]:
# In this tutorial we will use ./preprocessing_script.sh instead of mentioned above, because we want to have fixed path to cache dir

In [4]:
os.system("chmod a+x preprocessing_script.sh")
os.system("./preprocessing_script.sh")

[2022-02-02 17:35:59,862][root][INFO] - .......... Caching model and tokenizer ..........


2022-02-02 17:36:00.603632: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not in

[2022-02-02 17:36:07,326][root][INFO] - ========== Model distilbert-base-uncased successfully cached ==========
[2022-02-02 17:36:14,022][root][INFO] - ========== Tokenizer distilbert-base-uncased successfully cached ==========
[2022-02-02 17:36:14,023][root][INFO] - .......... Caching cached dataset ..........
[2022-02-02 17:36:15,627][datasets.builder][WARNING] - Using custom data configuration default
[2022-02-02 17:36:15,666][datasets.builder][WARNING] - Reusing dataset ag_news (../../../workdir/cache/test_data/data/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
[2022-02-02 17:36:15,712][root][INFO] - ========== Dataset ag_news successfully cached ==========


100%|██████████| 2/2 [00:00<00:00, 46.59it/s]

[2022-02-02 17:36:17,020][root][INFO] - .......... Caching metrics ..........
[2022-02-02 17:36:18,382][root][INFO] - ========== Main metric accuracy successfully cached ==========
[2022-02-02 17:36:19,478][root][INFO] - ========== Additional metric f1 successfully cached ==========



Error executing job with overrides: ['data.dataset_name=ag_news', 'acquisition_model.name=distilbert-base-uncased', 'cache_model_and_dataset=True', 'cache_dir=../../../workdir/cache/test_data']
Traceback (most recent call last):
  File "../../active_learning/utils/cache_all_necessary_files.py", line 40, in main
    rmtree(auto_generated_dir)
  File "/home/jovyan/.imgenv-free-cpu-0/lib/python3.7/shutil.py", line 498, in rmtree
    onerror(os.rmdir, path, sys.exc_info())
  File "/home/jovyan/.imgenv-free-cpu-0/lib/python3.7/shutil.py", line 496, in rmtree
    os.rmdir(path)
OSError: [Errno 39] Directory not empty: '/home/jovyan/active_learning/examples/run_cluster_job/workdir/run_active_learning/2022-02-02/17-35-55_42_distilbert_base_uncased'

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.


256

# Build image

Before running tasks, one could build an image using available base docker images. List of base images could be found here: https://mlspace.aicloud.sbercloud.ru/mlspace/datahub-catalog.

In [5]:
# List all available images - if you created images before, you can reuse one of them
client_lib.get_images()

['cr.msk.sbercloud.ru/728d63df-4bf7-4aaf-ba85-e2e7ceb9feea/job-custom-image-b07219:latest',
 'cr.msk.sbercloud.ru/728d63df-4bf7-4aaf-ba85-e2e7ceb9feea/job-custom-image-8bd99f:latest',
 'cr.msk.sbercloud.ru/728d63df-4bf7-4aaf-ba85-e2e7ceb9feea/job-custom-image-6f2439:latest',
 'cr.msk.sbercloud.ru/728d63df-4bf7-4aaf-ba85-e2e7ceb9feea/job-custom-image-8c76b2:latest',
 'cr.msk.sbercloud.ru/728d63df-4bf7-4aaf-ba85-e2e7ceb9feea/job-custom-image-07cf30:latest',
 'cr.msk.sbercloud.ru/728d63df-4bf7-4aaf-ba85-e2e7ceb9feea/job-custom-image-576f48:latest',
 'cr.msk.sbercloud.ru/728d63df-4bf7-4aaf-ba85-e2e7ceb9feea/job-custom-image-897c49:latest',
 'cr.msk.sbercloud.ru/728d63df-4bf7-4aaf-ba85-e2e7ceb9feea/job-custom-image-8e2201:latest',
 'cr.msk.sbercloud.ru/728d63df-4bf7-4aaf-ba85-e2e7ceb9feea/job-custom-image-b02978:latest']

In [14]:
# If not, you can create your own image
job = client_lib.ImageBuildJob(
    from_image="cr.msk.sbercloud.ru/aicloud-base-images/cuda10.2:0.0.25",  # base image
    requirements_file="/home/jovyan/active_learning/requirements.txt",  # requirements file
)

job.submit()

'ImageBuildJob "{\'image\': \'cr.msk.sbercloud.ru/728d63df-4bf7-4aaf-ba85-e2e7ceb9feea/job-custom-image-2368f0\', \'name\': \'image-build-job-jp9t6\', \'status\': \'ok\'}" created'

In [15]:
job.new_image  # image ID, used as an argument for running task

'cr.msk.sbercloud.ru/728d63df-4bf7-4aaf-ba85-e2e7ceb9feea/job-custom-image-2368f0'

In [60]:
job.logs()  # show logs of building container

INFO[0005] Resolved base name cr.msk.sbercloud.ru/aicloud-base-images/cuda10.2:0.0.25 to cr.msk.sbercloud.ru/aicloud-base-images/cuda10.2:0.0.25 
INFO[0005] Resolved base name cr.msk.sbercloud.ru/aicloud-base-images/cuda10.2:0.0.25 to cr.msk.sbercloud.ru/aicloud-base-images/cuda10.2:0.0.25 
INFO[0005] Retrieving image manifest cr.msk.sbercloud.ru/aicloud-base-images/cuda10.2:0.0.25 
INFO[0008] Retrieving image manifest cr.msk.sbercloud.ru/aicloud-base-images/cuda10.2:0.0.25 
INFO[0009] Built cross stage deps: map[]                
INFO[0009] Retrieving image manifest cr.msk.sbercloud.ru/aicloud-base-images/cuda10.2:0.0.25 
INFO[0009] Retrieving image manifest cr.msk.sbercloud.ru/aicloud-base-images/cuda10.2:0.0.25 
INFO[0009] Unpacking rootfs as cmd RUN if [ -e /context/requirements.txt ]; then  pip install --user --no-cache -r /context/requirements.txt; fi requires it. 
INFO[0319] Taking snapshot of full filesystem...        
INFO[0327] RUN if [ -e /context/requirements.txt ]; then  p

# Run job (it runs only .py files, so we wrap our .sh script into .py)

Limitations:
1. Better to use with .py files
2. No internet connection, so you have to load and cache models/datasets/etc

We also could create a Job with bash script, but we have to add several parameters.

NOTE: for now, better to use .py scripts, because .sh scripts will be runned in parallel mode in case of multiple nodes/instances

For now there are several additional params in Job class that doesn't mentioned in official docs, but could be useful for our tasks:
1. type - type of script, should be "binary" for bash scripts
2. region - region of instance, if set to "A100", will create instances with A100

In [191]:
# create a job
# this .sh script just print list of files in current directory
test_run_sh = client_lib.Job(
    base_image=job.new_image,  # ID of your custom image or one of the base images
    script="bash ./active_learning/examples/run_cluster_job/test.sh",  # script to run
    n_workers=1,
    n_gpus=2,
    type="binary",  # for .sh scripts
    warm_cache=False,
)

In [168]:
# create a job
test_run = client_lib.Job(
    base_image=job.new_image,  # ID of your custom image or one of the base images
    script="./active_learning/examples/run_cluster_job/test.py",  # script to run
    n_workers=1,
    n_gpus=2,
    warm_cache=False,
)

In [192]:
# add the job to the queue
test_run.submit()

'Job "lm-mpi-job-f7e6af43-643d-406f-b320-a8eb50c29ac3" created'

In [200]:
# list of all your jobs
# also shown at UI
client_lib.jobs()

2022-01-31T14:04:53Z : lm-mpi-job-08f2a950-191c-47a1-9a22-ef033cfca39e : Completed
2022-01-31T18:02:06Z : lm-mpi-job-135bdfb3-d611-411c-8358-7976c0a84f66 : Completed
2022-01-28T11:01:16Z : lm-mpi-job-14eb65bd-deca-448b-ab9b-09578b19c0fa : Completed
2022-01-28T11:35:55Z : lm-mpi-job-1734f7d1-0463-4876-ba68-de84b6d2e4fd : Completed
2022-01-31T17:42:12Z : lm-mpi-job-185e2300-9482-455f-838b-e4a202225b0d : Completed
2022-02-02T14:49:43Z : lm-mpi-job-1cfdb7d6-f990-49ea-a20a-1729ce1c82e5 : Completed
2022-01-31T11:20:08Z : lm-mpi-job-1e86c50d-8c01-4bfe-8f30-930ee61d9e13 : Completed
2022-01-31T14:52:54Z : lm-mpi-job-209f3943-159f-4268-ac0b-296f93f8c6f8 : Completed
2022-01-28T20:19:25Z : lm-mpi-job-2b03d9de-a5d9-40f2-9a69-572fb3c2cd94 : Completed
2022-02-02T15:06:50Z : lm-mpi-job-34ce5782-b460-491e-8a46-fc4ab53f0a43 : Completed
2022-01-31T17:54:34Z : lm-mpi-job-42f9be16-2697-4a30-af2f-4a6fb27e5a9b : Completed
2022-01-31T17:51:53Z : lm-mpi-job-4bf3d9ca-04c7-4fe3-af13-4ec5e336b13b : Completed
2022

In [ ]:
# show logs of the job
test_run.logs()

In [ ]:
# this command also show logs of a job, but get it by ID
client_lib.logs("lm-mpi-job-fbcfe043-6c97-44a5-b4d7-5ade9b223fed")

In [85]:
# you can kill the job, using job ID
client_lib.kill("lm-mpi-job-bff0fc50-49d6-49a8-a5ad-e01dc3ad477a")
# or using task
# test_run.kill()

'Job "lm-mpi-job-bff0fc50-49d6-49a8-a5ad-e01dc3ad477a" deleted'

In [98]:
client_lib.kill("lm-mpi-job-fe3ce5c3-9d1d-4f47-b1b2-4a82dc8c9253")

'Error 404 {"reason":"No such job. Job_name = lm-mpi-job-fe3ce5c3-9d1d-4f47-b1b2-4a82dc8c9253","status":"error"}\n'